In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error

# MySQL connection

In [ ]:
def load_data_from_mysql(host, user, password, database, tables):
    conn = pymysql.connect(host=host, user=user, password=password, database=database)
    dfs = []
    for table in tables:
        sql_query = f"SELECT * FROM {table}"
        df = pd.read_sql(sql_query, conn)
        dfs.append(df)
    conn.close()
    return pd.concat(dfs)

In [ ]:
# Since we have separate preprocess_data, train_and_evaluate_model functions for each table
# we'll define preprocess_data function and train_and_evaluate_model function for each table

In [ ]:
tables = ['table1', 'table2', 'table3', 'table4']  # Names of our 4 SQL tables
df = load_data_from_mysql(host='localhost', user='username', password='password', database='database_name', tables=tables)

for table in tables:
    X, y, cat_cols_idx = preprocess_data(df[df['table_name_column'] == table])
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=83, stratify=df[['Stage']])
    model, rmse_train, rmse_test = train_and_evaluate_model(X_train, X_test, y_train, y_test, cat_cols_idx)
    print(f"Table: {table}, RMSE score for train {round(rmse_train,2)} percentage points, and for test {round(rmse_test,2)} percentage points")

In [ ]:
def preprocess_data(df):
    cat_cols = df.select_dtypes(include=['object']).columns
    cat_cols_idx = [list(df.columns).index(c) for c in cat_cols]
    y = df[main_label].values.reshape(-1,)
    X = df.drop([main_label], axis=1)
    return X, y, cat_cols_idx


# CatBoost Regressor Model

In [ ]:
def train_and_evaluate_model(X_train, X_test, y_train, y_test, cat_cols_idx):
    # Create CatBoost pools
    train_pool = Pool(X_train, y_train, cat_features=cat_cols_idx)
    test_pool = Pool(X_test, y_test, cat_features=cat_cols_idx)
    
    # Model creation
    model = CatBoostRegressor(iterations=10000,
                              depth=4,
                              learning_rate=0.03,
                              verbose=0,
                              early_stopping_rounds=100,
                              loss_function='RMSE')
    
    # Model training
    model.fit(train_pool, eval_set=test_pool)
    
    # Make predictions
    y_train_pred = model.predict(train_pool)
    y_test_pred = model.predict(test_pool)
    
    # Calculate RMSE values
    rmse_train = mean_squared_error(y_train, y_train_pred, squared=False)
    rmse_test = mean_squared_error(y_test, y_test_pred, squared=False)
    
    return model, rmse_train, rmse_test

# Results

In [ ]:
results = {}
for table in tables:
    table_df = df[df['table_name_column'] == table]
    X, y, cat_cols_idx = preprocess_data(table_df)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=83, stratify=table_df[['Stage']])
    model, rmse_train, rmse_test = train_and_evaluate_model(X_train, X_test, y_train, y_test, cat_cols_idx)
    results[table] = {'RMSE_train': rmse_train, 'RMSE_test': rmse_test}
    
    
for table, result in sorted(results.items()):
    print(f"Table: {table}, RMSE score for train {round(result['RMSE_train'],2)} percentage points, and for test {round(result['RMSE_test'],2)} percentage points")    
